In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [ ]:
x_train = x_train[:1000]
y_train = y_train[:200]

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)



In [ ]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 10)


In [ ]:
# Attention Module
def attention_block(input_tensor, skip_tensor, filters):
    # 1x1 convolution on skip connection
    g = layers.Conv2D(filters, kernel_size=1, strides=1, padding='same')(skip_tensor)
    g = layers.BatchNormalization()(g)

    # 1x1 convolution on input tensor
    x = layers.Conv2D(filters, kernel_size=1, strides=1, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)

    # Add the two feature maps
    add = layers.Add()([g, x])
    add = layers.Activation('relu')(add)

    # 1x1 convolution to get attention coefficients
    psi = layers.Conv2D(1, kernel_size=1, strides=1, padding='same')(add)
    psi = layers.Activation('sigmoid')(psi)

    # Multiply attention coefficients with skip connection
    out = layers.Multiply()([skip_tensor, psi])
    return out

# U-Net with Attention
def unet_with_attention(input_shape=(32, 32, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)

    # Encoder (Downsampling)
    conv1 = layers.Conv2D(64, kernel_size=3, padding='same')(inputs)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Activation('relu')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, kernel_size=3, padding='same')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Activation('relu')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, kernel_size=3, padding='same')(pool2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Activation('relu')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    bottleneck = layers.Conv2D(512, kernel_size=3, padding='same')(pool3)
    bottleneck = layers.BatchNormalization()(bottleneck)
    bottleneck = layers.Activation('relu')(bottleneck)

    # Decoder (Upsampling) with Attention
    up1 = layers.UpSampling2D(size=(2, 2))(bottleneck)
    att1 = attention_block(up1, conv3, 256)
    concat1 = layers.Concatenate()([up1, att1])
    conv4 = layers.Conv2D(256, kernel_size=3, padding='same')(concat1)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Activation('relu')(conv4)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    att2 = attention_block(up2, conv2, 128)
    concat2 = layers.Concatenate()([up2, att2])
    conv5 = layers.Conv2D(128, kernel_size=3, padding='same')(concat2)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Activation('relu')(conv5)

    up3 = layers.UpSampling2D(size=(2, 2))(conv5)
    att3 = attention_block(up3, conv1, 64)
    concat3 = layers.Concatenate()([up3, att3])
    conv6 = layers.Conv2D(64, kernel_size=3, padding='same')(concat3)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Activation('relu')(conv6)

    # Output layer
    outputs = layers.Conv2D(num_classes, kernel_size=1, activation='softmax')(conv6)
    outputs = layers.GlobalAveragePooling2D()(outputs)  # Reduce spatial dimensions

    # Create model
    model = models.Model(inputs, outputs, name='UNet_Attention')
    return model



In [ ]:
# Build the model
model = unet_with_attention(input_shape=(32, 32, 3), num_classes=10)
model.summary()





Model: "UNet_Attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_17 (Conv2D)        │ (None, 32, 32, 64)     │          1,792 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_13    │ (None, 32, 32, 64)     │            256 │ conv2d_17[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_13             │ (None, 32, 32, 64)     │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 16, 16, 64)     │              0 │ activation_13[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 16, 16, 128)    │         73,856 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_14    │ (None, 16, 16, 128)    │            512 │ conv2d_18[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_14             │ (None, 16, 16, 128)    │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 8, 8, 128)      │              0 │ activation_14[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_19 (Conv2D)        │ (None, 8, 8, 256)      │        295,168 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_15    │ (None, 8, 8, 256)      │          1,024 │ conv2d_19[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_15             │ (None, 8, 8, 256)      │              0 │ batch_normalization_1… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 4, 4, 256)      │              0 │ activation_15[0][0]    │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_20 (Conv2D)        │ (None, 4, 4, 512)      │      1,180,160 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1

 Total params: 4,143,117 (15.80 MB)

 Trainable params: 4,138,509 (15.79 MB)

 Non-trainable params: 4,608 (18.00 KB)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train,
                    batch_size=8,
                    epochs=5,
                    validation_data=(x_test, y_test))

Epoch 1/5
3746/6250 ━━━━━━━━━━━━━━━━━━━━ 24:20 583ms/step - accuracy: 0.3417 - loss: 1.7863

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')